In [ ]:
import ultralytics
ultralytics.checks()

import torch
print(torch.backends.mps.is_available())

import os
os.getcwd()

import time

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="1SxC607xMtZgUVxVWOiL")
# project = rf.workspace("aifarms-zlefy").project("object-detection-horseradish-2")
# version = project.version(2)
# dataset = version.download("yolov8")

In [ ]:
# !pip install -U ultralytics

## Hyperparam with Abhi's script

In [ ]:
from ultralytics import YOLO

# best chosen YOLO model
model = YOLO("Horseradish_8n_200epochs.pt")  

# search space
search_space = {
    # "lr0": [1e-5, 1e-1],        # initial learning rate
    "batch_size": [2, 4, 8, 16],
    "epochs": [100, 150, 200],
    "optimizer": ['SGD', 'AdamW']
}
 
start = time.time()

data_path = "/home/sunoj/Documents/Abhi_paper/filtered_dataset/data.yaml"

# Tuning the model
model.tune(
    data=data_path,
    # lr0 = 0.00975,
    degrees = 0.0,
    iterations=10,
    space=search_space,
    plots=False,
    save=False,
    val=False,
    device = 'cuda'
)

end = time.time()

print(f"Time taken for tuning: {end - start} seconds")

## Tuning on Nov 6, 2025

In [ ]:
from ultralytics import YOLO

# Initialize the YOLO model
best_model = YOLO("/home/sunoj/Documents/Abhi_paper/Horseradish_8n_200epochs.pt")

start = time.time()

search_space = {
    "lr0": (1e-5, 1e-1),
    "momentum": (0.6, 0.99),
    "weight_decay": (0.0, 0.001),
    # "batch_size": (2, 4, 8, 16),
    # "degrees": (0.0, 45.0),
}

data_path = "/home/sunoj/Documents/Abhi_paper/filtered_dataset/data.yaml"

# Tune hyperparameters on horseradish dataset for 50 epochs
best_model.tune(data=data_path, 
           epochs=50,
           space=search_space, 
           iterations=100, 
           optimizer="AdamW", 
           plots=False, 
           save=False, 
           val=False
           )

end = time.time()

print(f"Time taken for tuning: {end - start} seconds")

## Inference of best model

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

# load trained model
model = YOLO("best_models/Horseradish_8n_200epochs.pt")

# single image path
img = "DJI_20250612151412_0220_D_JPG.rf.f81f4152522994408c01278baae4fe77.jpg".  # Benchy data

# run inference
results = model(img)

# plotting
plt.figure(figsize=(12, 10))
plt.imshow(results[0].plot())
plt.axis('off')
plt.show()

In [ ]:
from ultralytics import YOLO

model = YOLO("best_models/Horseradish_8n_200epochs.pt")

# print(model.model)          # architecture
print(model.model.yaml)     # model structure config


## Hyperparam on Nov 8, 2025 - Optuna bayesian optimization

In [ ]:
# !pip install optuna

In [4]:
import optuna
from ultralytics import YOLO

# your dataset yaml OR data dict
DATA = "filtered_dataset/data.yaml"   # replace with your dataset yaml

# your trained weights
WEIGHTS = "best_models/Horseradish_8n_200epochs.pt"


def objective(trial):

    # ----- suggested hyperparameters -----
    lr0           = trial.suggest_float("lr0", 1e-4, 3e-3, log=True)
    lrf           = trial.suggest_float("lrf", 0.01, 0.3)
    momentum      = trial.suggest_float("momentum", 0.80, 0.97)
    weight_decay  = trial.suggest_float("weight_decay", 5e-5, 1e-3, log=True)
    warmup_epochs = trial.suggest_int("warmup_epochs", 0, 3)

    box = trial.suggest_float("box", 3.0, 10.0)
    cls = trial.suggest_float("cls", 0.2, 2.0)
    dfl = trial.suggest_float("dfl", 0.2, 3.0)

    # translate = trial.suggest_float("translate", 0.0, 0.2)
    # scale     = trial.suggest_float("scale", 0.5, 1.5)

    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam", "AdamW"])


    # ----- load base model -----
    model = YOLO(WEIGHTS)

    # ----- train: short schedule for speed -----
    result = model.train(
        data=DATA,
        epochs=5,       # keep small for tuning
        lr0=lr0,
        lrf=lrf,
        momentum=momentum,
        weight_decay=weight_decay,
        warmup_epochs=warmup_epochs,
        box=box,
        cls=cls,
        dfl=dfl,
        # translate=translate,
        # scale=scale,
        optimizer=optimizer,
        verbose=False
    )

    # get metrics dictionary
    metrics = result.results_dict

    # segmentation: use metrics/seg_50
    if "metrics/seg_50" in metrics:
        score = metrics["metrics/seg_50"]

    # detection: metrics/mAP50(B)
    elif "metrics/mAP50(B)" in metrics:
        score = metrics["metrics/mAP50(B)"]

    else:
        # if missing, penalize
        score = 0

    return score


# create study
study = optuna.create_study(direction="maximize")

# run optimization
study.optimize(objective, n_trials=20)   # increase trials if needed

# print result
print("\nbest score:", study.best_value)
print("best params:", study.best_params)


[I 2025-11-08 19:44:03,148] A new study created in memory with name: no-name-bc080245-db75-4c18-be9c-14857d5119b1


Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.46202516090474, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.4814736676578405, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=0.7426894817226803, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002151781018028792, lrf=0.07863804719320737, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=best_models/Horseradish_8n_200epochs.pt, momentum=0.8487652159449941, mosaic=1.0, multi_scale=False, name=

[I 2025-11-08 19:44:25,245] Trial 0 finished with value: 0.9087825998366387 and parameters: {'lr0': 0.002151781018028792, 'lrf': 0.07863804719320737, 'momentum': 0.8487652159449941, 'weight_decay': 5.1348982221808096e-05, 'warmup_epochs': 0, 'box': 8.46202516090474, 'cls': 0.4814736676578405, 'dfl': 0.7426894817226803, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.9087825998366387.


Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.73474414078752, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5168247665901822, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.504190918308797, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002290445280176298, lrf=0.28855291907197084, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=best_models/Horseradish_8n_200epochs.pt, momentum=0.9675915445729226, mosaic=1.0, multi_scale=False, name=t

[I 2025-11-08 19:44:47,343] Trial 1 finished with value: 0.9453776308098065 and parameters: {'lr0': 0.002290445280176298, 'lrf': 0.28855291907197084, 'momentum': 0.9675915445729226, 'weight_decay': 0.00013248751289936472, 'warmup_epochs': 2, 'box': 8.73474414078752, 'cls': 0.5168247665901822, 'dfl': 1.504190918308797, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.9453776308098065.


...

Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.468096685688518, cache=False, cfg=None, classes=None, close_mosaic=10, cls=1.0539851928136237, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=2.496728942639565, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0009082840338937656, lrf=0.16676023165344367, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=best_models/Horseradish_8n_200epochs.pt, momentum=0.9480737573587704, mosaic=1.0, multi_scale=False, name

[I 2025-11-08 19:51:26,790] Trial 18 finished with value: 0.9349772977627178 and parameters: {'lr0': 0.0009082840338937656, 'lrf': 0.16676023165344367, 'momentum': 0.9480737573587704, 'weight_decay': 0.00038428838361223973, 'warmup_epochs': 0, 'box': 8.468096685688518, 'cls': 1.0539851928136237, 'dfl': 2.496728942639565, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.9453776308098065.


Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.349990646022059, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.38886532279713637, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.4591241600948315, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0028732841058607453, lrf=0.22780866865423208, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=best_models/Horseradish_8n_200epochs.pt, momentum=0.8904564810320523, mosaic=1.0, multi_scale=False, na

[I 2025-11-08 19:51:54,617] Trial 19 finished with value: 0.930578540149068 and parameters: {'lr0': 0.0028732841058607453, 'lrf': 0.22780866865423208, 'momentum': 0.8904564810320523, 'weight_decay': 0.0005437200873140366, 'warmup_epochs': 2, 'box': 8.349990646022059, 'cls': 0.38886532279713637, 'dfl': 1.4591241600948315, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.9453776308098065.



best score: 0.9453776308098065
best params: {'lr0': 0.002290445280176298, 'lrf': 0.28855291907197084, 'momentum': 0.9675915445729226, 'weight_decay': 0.00013248751289936472, 'warmup_epochs': 2, 'box': 8.73474414078752, 'cls': 0.5168247665901822, 'dfl': 1.504190918308797, 'optimizer': 'SGD'}


## Best parameters ran on Alienware

best score: 0.9419566807511106
best params: {'lr0': 0.0021886502060010807, 'lrf': 0.09102237299267574, 'momentum': 0.8746759339954618, 'weight_decay': 0.0003893275112975199, 'warmup_epochs': 2, 'box': 5.965853683536987, 'cls': 0.7347583795428045, 'dfl': 0.7083788852592507, 'optimizer': 'SGD'}